In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn import linear_model

dataset = pd.read_csv('data_train.csv')


In [2]:
# Fac drop la coloanele fnlgwt si Unnamed: 0, deoarece nu inteleg ce ele inseamna,ar putea afecta negativ rezultatul. Dar din 
# statistic point of view, e gresit, ca ar trebui sa consideram toate informatia avalabila
dataset.drop(["fnlwgt", "Unnamed: 0"], axis = 1, inplace = True) 

In [3]:
# Orice coloana categorica transform in dummy variable, dar si separ labelul in o coloana
targetData  = dataset['label']
dummy = pd.get_dummies(dataset, columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship','race', 'sex', 'native-country'])
total_data=dummy



In [4]:
# Pentru a incepe PCA trebuie de scos LABEL COLUMN
forpca=total_data
forpca.drop(["label"], axis = 1, inplace = True) 
forpca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Columns: 107 entries, age to native-country_ Yugoslavia
dtypes: int64(5), uint8(102)
memory usage: 4.0 MB


In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [6]:
# Aleg numarul de componente 70, ca varianta explicata sa fie aproximativ 80% 


standardizedFeatures= StandardScaler().fit_transform(forpca)

cov_mat = np.cov(standardizedFeatures.T)

eig_vals, eig_vecs = np.linalg.eig(cov_mat)

pca = PCA(n_components = 70).fit(standardizedFeatures)
print(pca.explained_variance_ratio_)


[0.04236209 0.0280058  0.02434848 0.02286331 0.02173486 0.0178335
 0.01620675 0.01487278 0.01395934 0.01323255 0.0129393  0.01227218
 0.01204797 0.01161648 0.01138297 0.01134595 0.01119837 0.01087002
 0.01073679 0.01064251 0.01047023 0.01040135 0.01036796 0.01023388
 0.01014001 0.01011789 0.01005533 0.00997971 0.00989889 0.00988114
 0.00987314 0.00978945 0.00971342 0.00967416 0.00967047 0.00964095
 0.00960816 0.00957059 0.00954942 0.00950202 0.00943148 0.00942415
 0.00940962 0.00937204 0.00936953 0.00936257 0.00935502 0.00934383
 0.00933177 0.00932373 0.0093169  0.00930287 0.00929274 0.00928134
 0.00926183 0.00922394 0.00921727 0.00921381 0.00918586 0.00917356
 0.00914861 0.00908924 0.00907907 0.00901956 0.00899956 0.00896594
 0.00894583 0.00888775 0.0088275  0.00875921]


In [7]:
# Aici avem un mare numar de variabile categorice. Dar si mai mult PCA nu se foloseste pentru categorical variabile, dar pentru 
# continue. Conform " https://en.wikipedia.org/wiki/Multiple_correspondence_analysis " PCA este inadequate in aceasta situatie
# si trebuie de folosit multiple correspondence analysis
pca.explained_variance_ratio_.sum() 

0.8045262982807218

In [8]:
pcaFeatures = pd.DataFrame(pca.transform(standardizedFeatures))

In [9]:
pcaFeatures

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,-4.088408,2.062500,2.046235,1.124859,0.376569,-1.163096,-0.627706,4.282365,1.603677,-0.731482,...,-1.438456,-3.229020,1.555152,2.019745,6.696658,1.525767,4.440208,2.754598,1.395334,-1.712111
1,-2.061484,3.188162,2.888375,-2.470671,1.394358,0.838017,-0.306600,1.679881,-1.721759,1.600877,...,-2.105215,-2.432241,-0.127833,-2.053000,-1.831212,3.204925,-2.205999,-0.431586,-3.606427,1.748551
2,-0.503466,-0.366181,-1.102315,0.053638,1.042795,-0.626469,-3.003873,1.904721,1.844658,-1.334724,...,0.859948,-1.123388,-0.049048,-0.775348,-0.917855,0.844737,0.264879,0.463983,1.418808,-0.483360
3,2.880376,-0.042299,-0.460430,0.878847,0.039409,-0.135614,-0.159263,-0.018506,2.242954,1.997480,...,0.099051,-0.004350,0.301192,0.279167,-0.328439,-0.077542,0.060758,-0.020833,-0.314934,-0.146391
4,2.409943,0.007204,-0.196019,1.290672,0.035634,-1.389668,-1.139191,2.444764,1.732783,-1.199443,...,0.852835,-1.160455,0.202470,-0.782069,-0.987471,0.665441,0.446261,0.479528,1.317895,-0.155749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29299,-3.585419,1.422494,0.068389,-0.389059,-0.654630,1.737600,1.089837,0.846864,-0.596271,-1.490272,...,0.192249,0.086380,-0.293184,-1.146063,-1.131153,0.662271,-0.672802,0.342360,-2.538107,-1.591390
29300,-1.320342,-3.000878,-0.541925,-0.298802,1.360928,0.074823,0.102550,1.232700,0.891471,0.726458,...,-0.527819,-0.075068,-0.513654,0.787709,-0.060254,0.373903,-0.546492,0.625698,-2.361935,0.230669
29301,-0.181224,-1.966444,-0.102030,-0.490481,-0.719051,0.491004,4.039517,0.136187,-0.180796,0.298248,...,0.560439,-0.394008,0.124642,0.021917,0.142540,-0.025800,-0.157084,-0.251465,0.308504,-0.981773
29302,1.693952,0.214615,0.117042,4.755605,3.334135,-0.026317,0.610663,-1.210664,-0.970244,1.252539,...,0.126672,-0.032851,-0.047230,-0.203165,-0.137628,-0.228875,0.137517,-0.033222,0.062321,-0.118986


In [10]:
# Se incepe regresia logica
label = pd.get_dummies(targetData, columns = ['label'], drop_first=True)


In [11]:
# are loc logistic regression
import statsmodels.api as sm
logit = sm.Logit(label, pcaFeatures)
result = logit.fit()

Optimization terminated successfully.
         Current function value: inf
         Iterations 7


C:\Users\portc\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1789: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


In [12]:
# we get a nice little summary
print(result.summary())

C:\Users\portc\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\portc\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


                           Logit Regression Results                           
Dep. Variable:                   >50K   No. Observations:                29304
Model:                          Logit   Df Residuals:                    29234
Method:                           MLE   Df Model:                           69
Date:                Wed, 04 Dec 2019   Pseudo R-squ.:                     inf
Time:                        04:00:12   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
0              0.5377      0.008     68.048      0.000       0.522       0.553
1             -0.4121      0.010    -42.926      0.000      -0.431      -0.393
2              0.1179      0.009     13.171      0.0

In [13]:
control=result.predict(pcaFeatures)
print(control)

0        0.181696
1        0.151609
2        0.147503
3        0.695241
4        0.479667
           ...   
29299    0.170818
29300    0.528046
29301    0.768210
29302    0.671155
29303    0.513783
Length: 29304, dtype: float64


In [14]:
s=0
for i in control:
    if i>0.5: control[s]=1
    else: control[s]=0
    s=s+1

In [15]:
control1=control.to_frame()

In [16]:
mars = pd.concat([control1, label], axis=1, join='inner')

In [17]:
mars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 2 columns):
0        29304 non-null float64
 >50K    29304 non-null uint8
dtypes: float64(1), uint8(1)
memory usage: 257.7 KB


In [18]:
import numpy as np
deci=mars[0]-mars[' >50K']
s=0
for i in np.arange(0,29304,1):
    if deci[i]!=0: s=s+1
print(s)

8029


In [19]:
# Deci numarul de succese cu cut-off de 0.5, este 29303-7925/29303 ~73%
# Adica acest logit imi da 73% sansa de prezicere


In [20]:

test = pd.read_csv('test.csv')

test.drop(["fnlwgt", "Unnamed: 0"], axis = 1, inplace = True) 

dummy2 = pd.get_dummies(test, columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship','race', 'sex', 'native-country'])
total_data2=dummy2

forpca2=total_data2
forpca2.drop(["label"], axis = 1, inplace = True) 

standardizedFeatures2= StandardScaler().fit_transform(forpca2)

Y= np.dot(standardizedFeatures2,eig_vecs)


control2=result.predict(Y[:,0:70])

s2=0
for i in control2:
    if i>0.5: control2[s2]=1
    else: control2[s2]=0
    s2=s2+1
    
### Printez rezultatele prezise, ele sunt salvate in array-ul control2
print(control2), print(len(control2))


FileNotFoundError: [Errno 2] File b'test.csv' does not exist: b'test.csv'

In [ ]:


# Penultimul paragraf contine, partea de program ce dat un dataset cu nume test.csv, va da output o coloana cu prezicerile, daca 
# a castigat 50k sau nu.




# Din pacate nu pot da o recomandare. Astfel de o recomandare mult depinde de matricea de eigenvalues care va da compozitiile 
# la features noi, dar si depinde coeficienti de la ecuatia logistica estimata. 
# Insa acest attempt de recomandare este si descurajat de faptul ca PCA nu este o metoda buna de folosit, cand
# sunt multe features categorice. 
# Conform " https://en.wikipedia.org/wiki/Multiple_correspondence_analysis " PCA este inadequate in aceasta situatie
# si trebuie de folosit multiple correspondence analysis.